# Fetching data from ArcGIS Online

### The ArcGIS Python API
The **ArcGIS Python API** is a powerful Python package for interacting with ArcGIS Online resources. ESRI provides ample documentation on this tool here: https://developers.arcgis.com/python/. Here, we'll use the API to <u>download MJB&A's exit data to a local shapefile</u>. 

We certainly won't have time to explore all this technology has to offer, but we will get at least a sample of what it can do. To explore further more effectively, I suggest you spend a bit of time familairizing yourself with the structure of the help documentation. 

<div class="alert alert-info">
Like many well-documented APIs, the documentation comes in three sections: a <A HREF=https://developers.arcgis.com/python/guide/>guide</a>, some <A HREF=https://developers.arcgis.com/python/sample-notebooks>example notebooks</a> and an <A HREF=https://developers.arcgis.com/python/api-reference/>API reference page</a>. I recommend starting by perusing the guide, to get a feel for just what the API's capabilities are, then perhaps run through some examples and tinker with them. Use the API reference to look up specific what specific programming objects can do. And don't forget that Googleing/StackExchange can be useful resources as well...
<div>

### Finding content with the ArcGIS Python API
One of the key features of the ArcGIS Python API is its ability to find resources located on ArcGIS Online. Previously, we examined how we can search ArcGIS Online resources from with ArcGIS Pro; here we do a similar search using the API. However, when used from within Python, we have direct access to the data in our Python environment which can be useful. 

Here we will review the process for locating, retrieving, and saving a spatial data sets using the ArcGIS Python API. A more in-depth tutorial on this topic can be found here: https://developers.arcgis.com/python/guide/accessing-and-creating-content, but we cover the basic workflow: 

#### Basic workflow:
1. Import the Python packages that give us the functionality we need
 * Here we also create a link to ArcGIS Online by creating a [GIS object](https://developers.arcgis.com/python/guide/using-the-gis/#Using-the-GIS)
* Search for content, storing the results of that search as a Python list of ArcGIS "**Items**"
* Select the specific **Item** we want to use and exploring its content
* Create a **FeatureLayer** object from that Item
* Query records from that FeatureLayer into a **FeatureSet** object
* Convert those records into a **SpatialDataFrame**
* Export that SpatialDataFrame to a local Shapefile

In [ ]:
#Import the arcgis GIS module - used to search AGOL for content
from arcgis import GIS

#Import the pandas package - used to work with our spatial dataframe
import pandas as pd

\[*You may get a warning about a deprecating package; you can ignore that...*]

In [ ]:
#Create an instance of the GIS object
gis = GIS()

### ♦ Searching for content
The ArcGIS Python API's [gis module](https://developers.arcgis.com/python/guide/the-gis-module/) allows us to interact with ArcGIS Online content. A subcomponent of this module is the `gis.content` object, referred to as the "Content Manager"; it provides the functionality to interact with content stored on ArcGIS Online. We'll use it here to search for content. 
* A fuller example of searching for content:  https://developers.arcgis.com/python/guide/accessing-and-creating-content/#Searching-for-content
* API reference for the `gis.content` object is [here](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#contentmanager)
 * [Scroll down](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.ContentManager.search) to find the API reference specific top the `gis.content.search()` function...
 * The `query` parameter reflects exactly what you might put into the Portal search box in ArcGIS Pro.
 * →*What is the default for the `max_items` parameter?*

In [ ]:
#Use the gis object to search content, storing the results as a variable called "results"
results = gis.content.search(query='NC exits 2019 owner:lukehellgren1')

In [ ]:
#Reveal how many "hits" we got
len(results)

---
<div class="alert alert-info">
► TO TRY: Try some different search terms and see how many hits you get. Try changing the <code>max_items</code> value to see if it is limiting the number of items returned.
</div>

In [ ]:
#Use the gis object to search content, storing the results as a variable called "results"
results2 = gis.content.search(query='',
                              max_items = 10)
len(results2)

---
Now, we'll examine the items returned in the "results" list..

In [ ]:
#Show the list of results
results

In [ ]:
#Extract the first item in the results list to the variable "theItem"
theItem = results[0]

In [ ]:
#Determine the data type of the "item" object
type(theItem)

We just create an ArcGIS "[item]()https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#item" object. This item is our link to a specific resource stored on some ArcGIS Online server. We can show a number of properties associated with this item. 
>An arcgis `item` refers to a specific AGOL service. Here it's a *feature* service, but it could also be a *map* service, an *image* service, or another service type. Different service types have different capabilities; the code we show below may be particular to feature services only. 

In [ ]:
#Show the item
theItem

In [ ]:
#Show the type of service this is
theItem.type

In [ ]:
#Show the item's title
theItem.title

In [ ]:
#Show the item's unique id
theItem.id

In [ ]:
#Show where the item is stored, i.e., its URL
print(theItem.url)

If you open the link above, you'll see the **REST endpoint** of the service which reveals information about the service includng what **layers** or **tables** (if any) are associated with the service. Here we see ours has one layer. We can also get this information via Python, as shown below. 

In [ ]:
#Show the layers associated with the item
theItem.layers

In [ ]:
#Extract the one (and only) layer into a new variable
layer = theItem.layers[0]
type(layer)

Now we have a new object - an ArcGIS [FeatureLayer](https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#featurelayer) object. 
>The `FeatureLayer` object is a link to an actual spatial dataset hosted on ArcGIS Online.

We are getting closer to the actual data object. The FeatureLayer's properties are accessed by calling it's 'properties' property. 'fields' is one property. Below we extract the set of fields associated with this FeatureLayer and then print each field's name.

In [ ]:
#Extract a list of fields from the layer's properties object
theFields = layer.properties.fields

In [ ]:
#Print all the field names
for f in theFields: 
    print (f.name)

The FeatureLayers object has a function called `get_unique_values()` which returns a list of unique values for a specific field..

In [ ]:
#Get unique values in a given field
layer.get_unique_values('State')

#### ♦FeatureLayer to FeatureSet via the `query()` command

In [ ]:
#Query a set of features from the layer
selected_features = layer.query("State = 'NC'")
type(selected_features)

In [ ]:
#How many feature have been returned?
len(selected_features)

#### ♦FeatureSet to Spatial Dataframe via the `sdf`  command
When we query the layer, it returns a new object, the arcgis [FeatureSet](https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#featureset). We can convert this to a [Spatially enabled dataframe](https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#spatialdataframe) which is much  has all the functionality of a Pandas dataframe *plus* some additional spatial abilities. 

In [ ]:
#Convert the FeatureSet to a SpatialDataFrame
sdfFeatures = selected_features.sdf
sdfFeatures.head()

By adding `.spatial` after the spatial dataframe object, we get access to its spatial capabilities. 

In [ ]:
#Show the spatial reference of the SpatialDataFrame
sdfFeatures.spatial.sr

In [ ]:
#Reveal the coordinates of the centroid of the dataset
sdfFeatures.spatial.centroid

#### ♦Saving the data to a local file
And finally we'll save the data to a local file. To do this, its useful to create folder into which the files will be sent. We can do this with Python's `os` module and its `mkdir` command.

In [ ]:
#Create a folder to hold the data
import os
os.mkdir("MJBA")

In [ ]:
#Export the data to the folder as a shapefile called "Exits.shp"
sdfFeatures.spatial.to_featureclass(location='MJBA/Exits.shp')

If you are working on a remote machine, you'll likely want to zip all the individual shapefile files into a single Zip archive for easy downloading. We can do this with the `shutil`'s `make_archive` funciton. 

In [ ]:
#Zip all the shapefiles into a zip file
from shutil import make_archive
make_archive('MJBA','zip','MJBA')

### Bonus: Preview of Mapping with the ArcGIS Python API

In [ ]:
#Create a map centered on Durham
theMap = gis.map('Durham, NC')
theMap

In [ ]:
#Plot the exits on the map
sdfFeatures.spatial.plot(theMap,
                         col='County',
                         renderer_type='u')